In [1]:
!wget https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv

--2022-05-03 02:40:54--  https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3858 (3.8K) [text/plain]
Saving to: ‘iris.csv.1’

iris.csv.1          100%[===================>]   3.77K  --.-KB/s    in 0s      

2022-05-03 02:40:54 (47.9 MB/s) - ‘iris.csv.1’ saved [3858/3858]



In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
iris = load_iris()
X = iris.data
y = iris.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [5]:
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.fit_transform(y_test.reshape(-1, 1)).toarray()

In [6]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        x = self.linear(x)
        return F.softmax(x, dim=1)

In [8]:
model = Model()

In [9]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [10]:
epochs = 100000
for epoch in range(epochs):
    epoch += 1
    
    # convert numpy array to torch Variable
    inputs = Variable(X_train)
    labels = Variable(y_train)
    
    # clear gradients w.r.t parameters
    optimizer.zero_grad()
    
    # forward to get output
    outputs = model(inputs)
    
    # calculate loss
    loss = criterion(outputs, labels)
    
    # getting gradients w.r.t parameters
    loss.backward()
    
    # updating parameters
    optimizer.step()
    
    if epoch % 10000 == 0:
        print('epoch {}, loss {}'.format(epoch, loss))

epoch 10000, loss 0.04997430369257927
epoch 20000, loss 0.029452119022607803
epoch 30000, loss 0.023273656144738197
epoch 40000, loss 0.020219873636960983
epoch 50000, loss 0.01835443079471588
epoch 60000, loss 0.01707368716597557
epoch 70000, loss 0.016127098351716995
epoch 80000, loss 0.015391167253255844
epoch 90000, loss 0.014797690324485302
epoch 100000, loss 0.01430581696331501


In [11]:
# test model
with torch.no_grad():
    inputs = Variable(X_test)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)

In [12]:
labels = Variable(y_test)
_, y_test_ = torch.max(labels.data, 1)

In [13]:
print("Accuracy:", metrics.accuracy_score(y_test_.numpy(), predicted.numpy()))

Accuracy: 0.9333333333333333
